In [1]:
#Set up workspace
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pysal
import statsmodels.api as sm
from pysal import W, lat2W
from pysal.cg.kdtree import KDTree

/Users/thomashill/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read in data to use 
all_listings = pd.read_csv('All_Listings_geocoded.csv')
sold_listings = pd.read_csv('Sold_Listings_geocoded.csv')

/Users/thomashill/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Concatenate long and lat coordinates into a new column
sold_listings['Coordinates'] = list(zip(sold_listings.DisplayX, sold_listings.DisplayY))

#Add a PPSF column
PPSF = sold_listings['SOLDPRICE']/sold_listings['SQFT']
sold_listings = sold_listings.assign(PPSF = PPSF)

#Get list of property types
prop_types = np.unique(sold_listings['TYPE'])

In [8]:
df_2016 = sold_listings[sold_listings['YEAR_ORIG']==2016]
df_2017 = sold_listings[sold_listings['YEAR_ORIG']==2017]


In [10]:
#Write a formula to find the PPSF of the k nearest neighbors of a given property

def find_neighbors_point(point,dataframe,num_neighbors):
    if len(dataframe)<1:
        PPSFs = []
        for neighbor in range(num_neighbors):
            PPSFs.append(np.nan)
    else:
        coordinates = list(dataframe['Coordinates'])
        tree = KDTree(coordinates, distance_metric='Arc', radius=pysal.cg.RADIUS_EARTH_KM)
        dists,neibs = tree.query((point), k=num_neighbors)

        PPSFs = []
        for neighbor in neibs:
            if np.isnan(neighbor):
                PPSF_value = np.nan
            else:
                row = dataframe.iloc[[neighbor]]
                PPSF_value = row['PPSF']
                PPSF_value = PPSF_value.values[0]
            PPSFs.append(PPSF_value)
    neighbor_values = pd.DataFrame({
        'PPSFs': PPSFs})
    neighbor_values = neighbor_values.transpose()
    
    
    return neighbor_values, dists

In [11]:
#Write a formula to find the k-nearest neighbors PPSF and dist for each point in an entire dataframe

def find_neighbors_df(df,num_neighbors=50):
    df_2016 = df[df['YEAR_ORIG']==2016]
    df_2017 = df[df['YEAR_ORIG']==2017]

    neighbor_PPSFs = []
    neighbor_dists = []
    for i in list(df_2017['Coordinates']):
        neighbors,dists = find_neighbors_point(i,df_2016,num_neighbors)
        neighbor_PPSFs.append(neighbors) 
        neighbor_dists.append(dists)
    
    added_columns = list(range(0, num_neighbors))
    
    added_columns_PPSF = [str(i)+'_PPSF' for i in added_columns]
    added_columns_dist = [str(i)+'_Dist' for i in added_columns]
    
    neighbor_PPSF_df = pd.concat(neighbor_PPSFs)
    neighbor_PPSF_df.columns = added_columns_PPSF
    neighbor_PPSF_df = neighbor_PPSF_df.reset_index()
    
    neighbor_dist_df = pd.DataFrame(neighbor_dists)
    neighbor_dist_df.columns = added_columns_dist
    neighbor_dist_df = neighbor_dist_df.reset_index()
        
    df_2017 = df_2017.reset_index()
    
    df_2017 = pd.concat([df_2017,neighbor_PPSF_df,neighbor_dist_df],axis=1)
    
    return df_2017

The above is a computationally intensive procedure. Since this is an O(n^2) operation, partition the dataset so that the calculation takes less time. 

In [355]:
MSA_cities = ['Abington',' Amesbury Town', 'Andover', 'Arlington', 'Ayer', 
              'Bellingham', 'Belmont', 'Beverly',' Boston', 'Boxford', 'Braintree Town', 'Bridgewater', 'Brockton', 'Brookline', 'Burlington',
              'Cambridge', 'Chelsea', 'Cochituate', 
              'Danvers', 'Dedham', 'Dover', 'Duxbury', 'East Pepperell', 'Essex', 'Everett'
              'Foxborough','Framingham','Franklin Town', 'Gloucester', 'Green Harbor-Cedar Crest', 'Groton',
              'Hanson', 'Haverhill', 'Hingham', 'Holbrook', 'Hopkinton', 'Hudson', 'Hull',
              'Ipswich',"Kingston",
              'Lawrence','Lexington', 'Littleton Common', 'Lowell', 'Lynn', 'Lynnfield', 
              'Malden', 'Marblehead', 'Marion Center', 'Marlborough', 'Marshfield', 'Marshfield Hills', 'Mattapoisett Center', 'Maynard',
              'Medfield','Medford','Melrose','Methuen Town','Middleborough Center','Millis-Clicquot','Milton',
              'Nahant','Needham','Newburyport','Newton','North Lakeville','North Pembroke','North Plymouth','North Scituate','Norwood',
              'Ocean Bluff-Brant Rock', 'Onset',
              'Peabody', 'Pepperell', 'Pinehurst', 'Plymouth',
              'Quincy',
              'Randolph', 'Reading', 'Revere', 'Rockport', 'Rowley', 
              'Salem', 'Salisbury', 'Saugus', 'Scituate', 'Sharon', 'Shirley', 'Somerville', 'South Duxbury', 'Southfield', 'Stoneham', 'Swampscott',
              'The Pinehills', 'Topsfield', 'Townsend', 
              'Wakefield', 'Walpole', 'Waltham', 'Wareham Center', 'Watertown Town', 'Wellesley', 'West Concord', 'West Wareham',
              'Weweantic', 'Weymouth Town', 'White Island Shores', 'Wilmington', 'Winchester',  'Winthrop Town', 'Woburn']

non_MSA_cities = list(set(list(np.unique(sold_listings['CITY']))) - set(MSA_cities))

df_MSA = sold_listings.loc[sold_listings['CITY'].isin(MSA_cities)]
df_non_MSA = sold_listings.loc[sold_listings['CITY'].isin(non_MSA_cities)]

In [362]:
num_neighbors = 50

full_type_dfs = []
for prop_type in prop_types:
    type_df = df_MSA[df_MSA['TYPE']==prop_type]
    new_df = find_neighbors_df(type_df,num_neighbors)
    full_type_dfs.append(new_df)
MSA_neighbors_df_2017 = pd.concat(full_type_dfs)

In [363]:
num_neighbors = 50

full_type_dfs = []
for prop_type in prop_types:
    type_df = df_non_MSA[df_non_MSA['TYPE']==prop_type]
    new_df = find_neighbors_df(type_df,num_neighbors)
    full_type_dfs.append(new_df)
non_MSA_neighbors_df_2017 = pd.concat(full_type_dfs)

In [370]:
neighbors_df_2017 = pd.concat([MSA_neighbors_df_2017,non_MSA_neighbors_df_2017])

In [371]:
#Replace the PPSF's with the predicted prices based on those PPSF's (i.e., PPSF*row property's SF)

added_columns = list(range(0, num_neighbors))
added_columns_PPSF = [str(i)+'_PPSF' for i in added_columns]

for i in added_columns_PPSF:
    predicted_price = neighbors_df_2017[i]*neighbors_df_2017['SQFT']
    neighbors_df_2017[i] = predicted_price

In [376]:
#Save the above for future use
neighbors_df_2017.to_csv("Neighbors_df_2017.csv")
